In [2]:
# Define measurement names
MEASUREMENTS_PARASTERNAL_LONG_AXIS_VIEW = ['Diám raíz Ao', \
    'Diám. Ao asc.', 'Diám TSVI', 'Dimensión AI']
# POTENTIAL_MEASUREMENTS_PARASTERNAL_LONG_AXIS_VIEW = ['Diám TSVD', \
#      'DVItd', 'DVIts', 'SIVtd', 'PPVItd']
# Note: Removed 'Diam TSVD' as a measurement which would classify
# a view as PLAX as Antonio is unsure of this, 2019_09_07
# This change disqualifies 650 frames from being considered PLAX
POTENTIAL_MEASUREMENTS_PARASTERNAL_LONG_AXIS_VIEW = [ \
     'DVItd', 'DVIts', 'SIVtd', 'PPVItd']
MEASUREMENTS_APICAL_4_CHAMBER_VIEW = ['AVItd ap4', 'VTD(el-ps4)', \
     'VTD(MDD-ps4)', 'VTD 4C', 'AVIts ap4', 'VTS(el-ps4)', \
     'VTS(MDD-ps4)', 'VTS 4C', 'Vol. AI (MOD-sp4)']
MEASUREMENTS_APICAL_2_CHAMBER_VIEW = ['AVItd ap2', 'VTD(el-ps2)', \
    'VTD(MDD-ps2)', 'VTD 2C', 'AVIts ap2', 'VTS(el-ps2)', \
    'VTS(MDD-ps2)', 'VTS 2C', 'Vol. AI (MOD-sp2)']
ALL_MEASUREMENTS = MEASUREMENTS_PARASTERNAL_LONG_AXIS_VIEW + \
    POTENTIAL_MEASUREMENTS_PARASTERNAL_LONG_AXIS_VIEW + \
    MEASUREMENTS_APICAL_4_CHAMBER_VIEW + MEASUREMENTS_APICAL_2_CHAMBER_VIEW
MEASUREMENTS_END_DIASTOLIC = ['DVItd', 'SIVtd', 'PPVItd', \
'AVItd ap4', 'VTD(el-ps4)', 'VTD(MDD-ps4)', 'VTD 4C', 'AVItd ap2', \
'VTD(el-ps2)', 'VTD(MDD-ps2)', 'VTD 2C']
MEASUREMENTS_END_SYSTOLIC = ['DVIts', 'AVIts ap4', 'VTS(el-ps4)', \
    'VTS(MDD-ps4)', 'VTS 4C', 'AVIts ap2', 'VTS(el-ps2)', \
    'VTS(MDD-ps2)', 'VTS 2C']

In [3]:
io_clean = dbReadWriteClean()
io_views = dbReadWriteViews()

measurement_abstract_rpt_df = io_clean.get_table('measurement_abstract_rpt')
measgraphref_df = io_clean.get_table('a_measgraphref')
measgraphic_df = io_clean.get_table('a_measgraphic')

In [4]:
# Filter out unnecessary columns in each table
measgraphref_df = measgraphref_df[['studyidk', \
    'measabstractnumber', 'instanceidk', 'indexinmglist']]
measgraphic_df = measgraphic_df[['instanceidk', \
    'indexinmglist', 'frame']]
measurement_abstract_rpt_df = measurement_abstract_rpt_df[[\
    'studyidk', 'measabstractnumber', 'name']]

# Merge individual dataframes into one
merge_df = measgraphref_df.merge(measgraphic_df, on=[\
    'instanceidk', 'indexinmglist'])
merge_df = merge_df.merge(measurement_abstract_rpt_df, on=[\
    'studyidk', 'measabstractnumber'])

merge_df.shape

(823833, 6)

In [5]:
# df containing all frames for which we have measurements
filter_df = merge_df#[merge_df.name.isin(ALL_MEASUREMENTS)].copy()

filter_df['is_end_diastolic'] = filter_df['name'].isin(MEASUREMENTS_END_DIASTOLIC)
filter_df['is_end_systolic'] = filter_df['name'].isin(MEASUREMENTS_END_SYSTOLIC)

filter_df['is_plax'] = filter_df['name'].isin(MEASUREMENTS_PARASTERNAL_LONG_AXIS_VIEW)
filter_df['maybe_plax'] = filter_df['name'].isin(POTENTIAL_MEASUREMENTS_PARASTERNAL_LONG_AXIS_VIEW)
filter_df['is_a4c'] = filter_df['name'].isin(MEASUREMENTS_APICAL_4_CHAMBER_VIEW)
filter_df['is_a2c'] = filter_df['name'].isin(MEASUREMENTS_APICAL_2_CHAMBER_VIEW)

filter_df['view'] = ''
filter_df.loc[filter_df['is_plax']==True, 'view'] = 'plax'
filter_df.loc[filter_df['maybe_plax']==True, 'view'] = 'plax'
filter_df.loc[filter_df['is_a4c']==True, 'view'] = 'a4c'
filter_df.loc[filter_df['is_a2c']==True, 'view'] = 'a2c'

filter_df.shape

(823833, 13)

In [6]:
group_df = filter_df.groupby(['instanceidk', 'indexinmglist']).first()
group_df = group_df.drop(['measabstractnumber', 'name'], axis='columns')

group_df = group_df.reset_index()
(group_df.reset_index().groupby(['instanceidk', 'indexinmglist'])['view'].nunique().eq(1)==False).sum()
(group_df.reset_index().groupby('instanceidk')['view'].nunique().eq(1)==False).sum()

13198

In [7]:
is_instance_multiview = (group_df.reset_index().groupby('instanceidk')['view'].nunique().eq(1)==False).reset_index()

In [8]:
is_instance_multiview = is_instance_multiview.rename(index=str, columns={"view": "is_multiview"})

In [9]:
group_df = group_df.merge(is_instance_multiview, on='instanceidk')

In [10]:
frames_with_views_df = group_df#.merge(is_instance_multiview, on='instanceidk')
frames_with_views_df = frames_with_views_df.drop(['is_plax', 'maybe_plax', 'is_a4c', 'is_a2c'], axis=1)

frames_with_views_df.head()

,instanceidk,indexinmglist,studyidk,frame,is_end_diastolic,is_end_systolic,view,is_multiview
0,57160,0,2882,0.0,False,False,,False
1,57160,1,2882,0.0,False,False,,False
2,57160,2,2882,0.0,False,False,,False
3,57160,3,2882,0.0,False,False,,False
4,57162,0,2882,5.0,False,False,plax,False


In [29]:
frames_with_views_df.shape

(486877, 8)

In [27]:
frames_with_views_df['is_multiview'].value_counts()
# number of frames with conflicts

False    429469
True      57408
Name: is_multiview, dtype: int64

In [31]:
frames_with_single_views_df = df1.drop(df1[(df1['is_multiview']==True)].index)
# here we drop all with multi_view

frames_with_single_views_df['is_multiview'].value_counts()

False    429469
Name: is_multiview, dtype: int64

In [22]:
frames_with_single_views_df['view'].value_counts()

        287774
plax     94601
a4c      28197
a2c      18897
Name: view, dtype: int64